# Physics 77, Lecture 3: Basics, Functions, Loops, Lists and Array

## Outline
- Recap
- Conditionals
- Loops


# Recap: Week 3

- Loop back: data representation, bits and bytes
- Composite Types (Lists, tuples, arrays)
- Functions
- Modules

### Number of bytes and precision of data types

In [1]:
import sys

Let's determine the sizes of basic types

In [2]:
sys.getsizeof(int)  # huh ?

408

In [3]:
sys.getsizeof(int(4)) # more realistic. 8 bytes for the data, 20 bytes of overhead

28

In [4]:
sys.maxsize   # this is a 64-bit integer, because I am running a 64-bit version of Python

9223372036854775807

In [5]:
2**63-1  # check

9223372036854775807

Interesting thing about Python is that it would allocate more bytes for data if needed, so there is no practical limit to the integer value

In [6]:
print(2**65-1)
print(type(2**65-1))

36893488147419103231
<class 'int'>


In [7]:
print(sys.getsizeof(2**65))
print(sys.int_info)

36
sys.int_info(bits_per_digit=30, sizeof_digit=4, default_max_str_digits=4300, str_digits_check_threshold=640)


Floating point numbers:

In [8]:
sys.getsizeof(float(1.0))   # apparently, also 24 bytes

24

Largest representable float

In [9]:
1.6e+308

1.6e+308

In [10]:
x=3e+308
print(x)

inf


In [11]:
5+7/180.+1.8e308/1.8e+308

nan

Smallest representable float

In [12]:
5e-324

5e-324

In [13]:
2e-324

0.0

Roundoff errors

In [20]:
a = 1e15-1
b = 1e15+1
print("Size of float:",sys.getsizeof(a))

d1 = (b**2-a**2)
d2 = (b-a)*(a+b)

print(d1,d2)

print("Ratio = ",d1/d2)

Size of float: 24
3940649673949184.0 4000000000000000.0
Ratio =  0.985162418487296


In [25]:
import decimal as d
d.getcontext().prec = 24  # set precision to 24 decimal places

a = d.Decimal(1e15)-1
b = d.Decimal(1e15)+1
print("Size of Decimal object:",sys.getsizeof(a))

d1 = (b**2-a**2)
d2 = (b-a)*(a+b)

print(d1,d2)
print("Ratio = ",d1/d2)

Size of Decimal object: 104
4.000000000E+15 4000000000000000
Ratio =  1


#### Sizes of composite data types (tuples, lists, arrays)

In [26]:
l = [1,2,4,5,6]  # 5*8=40 bytes of information
sys.getsizeof(l)

104

In [27]:
t = (1,2,4,5,6)
sys.getsizeof(t)

80

In [28]:
import numpy as np
N=10000
a = np.zeros(N,dtype=np.float64)
print(a)
print('size of a {0:d}-element array is {1:d} bytes, {2:3.1f} bytes/element'.format(N,sys.getsizeof(a),sys.getsizeof(a)/N))

[0. 0. 0. ... 0. 0. 0.]
size of a 10000-element array is 80112 bytes, 8.0 bytes/element


#### Beware of floating point comparisons !

In [29]:
i = 1
print(type(i))
print(1 == i)

<class 'int'>
True


In [30]:
x = 11.000000000000001/10
y = 1.1
x == y

True

In [31]:
(x-y)<1e-6   # control precision of the comparison, don't rely on automatic precision

True

In [32]:
import numpy as np

In [33]:
np.pi

3.141592653589793

In [34]:
np.pi == 3.14159265358979323846264338327950288419716939

True

In [35]:
3.14159265358979 == 3.141592653589793

False

In general, the precision of the arithmetic comparisons (==) is not guaranteed. Behavior in Python may be very different from other languages, may depend on OS, compilers, etc. It is considered bad practice to use == comparisons on floating point data. Preferably, you should check if the difference between the two numbers is within a certain precision:

In [36]:
x = 3.1415926
abs(x-np.pi)<1e-6

True

### Formatted output

Usually the data you manipulate has finite precision. You do not know it absolutely precisely, and therefore you should not report it with an arbitrary number of digits. One of the cardinal rules of a good science paper: round off all your numbers to the precision you know them (or care about) -- and no more ! 

#### Examples:

In [37]:
t = 45.34   # I only know 4 digits
print(t)   # OK, let Python handle it

45.34


That's actually pretty good -- Python remembered stored precision !
What happens if you now use x in a calculation ? 

In [38]:
print(np.sqrt(t))

6.73349834781297


Do we really know the output to 10 significant digits ? No ! So let's truncate it

In [39]:
t2 = 144.75
print('sqrt(t ) = {0:5.3f}, sqrt(t2) = {1:5.3f}'.format(np.sqrt(t),np.sqrt(t2)))
#print('sqrt(t2) = {0:5.2f}'.format(np.sqrt(t2)))

sqrt(t ) = 6.733, sqrt(t2) = 12.031


Another (deprecated) way to skin this cat:

In [40]:
print('sqrt(x) = %5.2f' % np.sqrt(x))
print('sqrt(y) = %5.2f' % np.sqrt(101))

sqrt(x) =  1.77
sqrt(y) = 10.05


In [41]:
print ('sqrt(x) = {0:3.2e}, x**2 = {1:4.2f}'.format(np.sqrt(x)*1000,x**2))
print ('x**2 = {1:4.2f}, sqrt(x) = {0:3.2e}'.format(np.sqrt(x),x**2))
print ('sqrt(x) = %3.2e, x**2 = %4.2f' % (np.sqrt(x), x**2))

sqrt(x) = 1.77e+03, x**2 = 9.87
x**2 = 9.87, sqrt(x) = 1.77e+00
sqrt(x) = 1.77e+00, x**2 = 9.87


String formatting: can specify the length of the field and justification. By default, the strings are left-justified, and if the length specifier exceeds the length of the string, the string is padded by white space

In [42]:
str = 'Jones, Bill'
print('My name is {0:20s} and I like {1:s}'.format(str,'pineapples'))

My name is Jones, Bill          and I like pineapples


To change justification, use "<" to left-justify, ">" to right-justify, and "^" to center:

In [43]:
print('My name is {0:<20s} and I like {1:s}'.format(str,'pineapples'))

My name is Jones, Bill          and I like pineapples


Another cool feature of `format()` function (in Python 3) is ability to label parameters by readable names instead of numbers

In [44]:
print ('x={x:4.2f}, sqrt(x) = {sqrt:4.2f}, x**2 = {square:4.2f}'.format(x=x,sqrt=np.sqrt(x),square=x**2))

x=3.14, sqrt(x) = 1.77, x**2 = 9.87


For more formatting options, see https://pyformat.info/

A tricky example: Treat '%' in the old-style formatting expression as an operator which has highest precedence. It is the operator that takes a string and another type, and returns a string. E.g. 

In [45]:
stringFormat = 'var = %5.3f'
print(stringFormat, type(stringFormat))

out = stringFormat % 2.5
print(out)
print(type(out))
print(stringFormat % 75)
print(stringFormat % 123.5)
print(stringFormat % 0.576)




var = %5.3f <class 'str'>
var = 2.500
<class 'str'>
var = 75.000
var = 123.500
var = 0.576


So far so good. But what if you want to do some operations inline, e.g.:

In [46]:
print('var = %5.3f' % 5./2)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

This does not work, because % takes the first number (5.) and converts it to formatted string; the string cannot be divided by an integer (2)
So again, use parentheses when in doubt ! 

In [47]:
print('var = %5.3f' % (5./2))

var = 2.500


## Conditionals

Conditionals are commands that are executed only if some condition is satisfied.
Beware! Indentation is important in Python. Note that it doesn't really matter how broad the indentation is.


**Example: Heaviside step function**

All the functions we considered so far were well behaved. But how do we code a step function in Python?

$y = f(x) =
\begin{cases}
0 \quad \text{if}\quad x<0 \\
1 \quad \text{if}\quad x\geq0
\end{cases}
$

In [70]:
def Theta( x ):

    res = 1
    if x < 0.:
        res = 0.
   
    
    print('res=',res)
    return res

xraw = input('Enter numerical value: ') # Ask the user to privide a value
print(type(xraw))   # beware ! In Python 3 this returns a string, which needs to be converted to int or float type
#x = eval(xraw)      # Also beware of potential security risks (buffer overflow)
print(Theta(float(xraw)))

Enter numerical value: 5
<class 'str'>
res= 1
1


An important thing to take into account is the indentation!

In [74]:
x=-500

if x < 0 :
    if x < -100 : 
        x=np.nan
        print('Very small')
        
    print ('This was a negative value')
    x = -x   # only executed for negative numbers

print (x)    # always executed

Very small
This was a negative value
nan


Sometimes you may want to do two different things:

In [76]:
sum = 10
xraw = input('Enter numerical value: ')
x=float(xraw)
if x < 0:
    print('In the negative branch')
    sum = sum - x
else :
    print('In the else branch')
    sum += x
print (sum)

Enter numerical value: -15
In the negative branch
25.0


And sometimes you may need to have several branches

In [79]:
value = 0 # this line is not needed
x = float(input('Enter numerical value: '))
if x > 5 :
    value = +3
elif x > 1 : # else if
    value = +2
elif x > -1 :
    value = 1
else :
    value = -2
    
print (value)

Enter numerical value: -10.976
-2


## Loops

### While

The while loop repeats an execution while (as long as) a condition is valid.

In [82]:
sum = 0
count = 0
while count < 10:
    sum += 10
    count += 1
    print (sum)
    
     
print (sum, count)

10
20
30
40
50
60
70
80
90
100
100 10


**Special keywords: break, continue, pass, else**

break:

In [86]:
sum = 0
count = 0
while sum > -100:
    sum += 10
    count += 1
    if count >= 6:
        break
        
        
    print(sum,count)
    
print (sum, count)

10 1
20 2
30 3
40 4
50 5
60 6


continue:

In [87]:
sum = 0
count = 0
while sum < 100000:
    sum += 10
    count += 1
    if count > 4 :
        continue
    print (sum)
    
print (sum, count)

10
20
30
40
100000 10000


else:

In [89]:
sum = 0
count = 0
while sum < 100:
    sum += 10
    count += 1
    if count >= 60:
        break
else:                                     # beware of indentation !!!
    print ("Finished without break")
    
    
print (sum, count)

Finished without break
100 10


In [90]:
sum = 0
count = 0
while sum < 100:
#    sum -= 10       # typo ! 
    sum += 10       # fixed typo ! 
    count += 1
    print(sum)
    
print (sum, count)

10
20
30
40
50
60
70
80
90
100
100 10


## For

The for loop is more conventional and repeats the execution for an index within a given range. This is similar to for() loop in C or other languages.

An equivalent syntax in C would be for(int i=0;i<10;i++) {}

In [92]:
list = range(0,10)
print(list[5])
print (len(list))

5
10


In [93]:
list = [11,22,33,77]

for i in list:    # loop from 0 to 10, not including 10, with step = 1
    print (i, i*2)

11 22
22 44
33 66
77 154


In [95]:
for i in range(0,10,3):   # loop from 0 to 10, not including 10, with step = 3
    print (i)
    x = np.sqrt(float(i)**3)
    print(x)

0
0.0
3
5.196152422706632
6
14.696938456699069
9
27.0


In [111]:
list = [1,2,3,4,7,111.,67.] # iterate over elements of the tuple
list.append(12)          # what happens here ? 
print('The list is',len(list),'elements long')
i=0
list2=list*2
print(len(list2))
for x in list:
    print(i,x,list2[i])
    print ('Element #',i,'is x=',x,'and its square is',x**2)
    i+=1

The list is 8 elements long
16
0 1 1
Element # 0 is x= 1 and its square is 1
1 2 2
Element # 1 is x= 2 and its square is 4
2 3 3
Element # 2 is x= 3 and its square is 9
3 4 4
Element # 3 is x= 4 and its square is 16
4 7 7
Element # 4 is x= 7 and its square is 49
5 111.0 111.0
Element # 5 is x= 111.0 and its square is 12321.0
6 67.0 67.0
Element # 6 is x= 67.0 and its square is 4489.0
7 12 12
Element # 7 is x= 12 and its square is 144


You can iterate over lists produced by other functions, e.g. a list of keys to a dictionary 

In [104]:
lastnames = {}                        # create a dictionary
lastnames['Billy'] = 'Jones'
lastnames['Johnny'] = 'Jones'
lastnames['Johnny'] = 'Baker'
#lastnames[5] = 'Foo'

#print(lastnames['Yury'])
lastnames['Yury'] = 'Kolomensky'

#list = sorted(lastnames.keys(),reverse=True)
#print (list)
for key in sorted(lastnames.keys(),reverse=True):          # iterate over elements of the dictionary
    print (key, lastnames[key])


Yury Kolomensky
Johnny Baker
Billy Jones


### Nesting and recursive functions

We have seen already a few examples of an if statement inside a while loop: this called nesting. Python sets no limit to nesting, i.e. you can have infinite statements and loops within each other.

In [105]:
def factorial(n):                # definition of the function
    value = 1
    for i in range(2,n+1):       # loop
        value *= i               # increment factorial 
        
    return value                 # return value

print ('factorial(10)=',factorial(10))
for i in range(1,5):
    print ('factorial(%d)=%2d' % (i,factorial(i)))

#print(factorial(1.1))

factorial(10)= 3628800
factorial(1)= 1
factorial(2)= 2
factorial(3)= 6
factorial(4)=24


In [109]:
def factRecursive(n):
    if n <=1:
        return 1
    
    return factRecursive(n-1)*n

print(factRecursive(1.1))

1.1


Here is a more elegant way to implement the function (recursive). It also has basic error handling

In [110]:
import numpy as np
def factRecursive(n):
    '''Computes n!, input: integer, output: integer'''
    if type(n)!=int:                     # these factorials defined only for integers
        return np.nan                    # return Not-a-number
    if n > 1:
        return n*factRecursive(n-1)      # THIS IS THE RECURSION!!
    elif n >= 0:
        return 1
    else:
        return -np.inf                 # return negative infinit
    
print (factRecursive(10))
print (factRecursive(-1))
print (factRecursive('Joe'))

x = factorial(5)   # old function still defined
y = x**2
print (y)

3628800
-inf
nan
14400
